In [1]:
import pandas as pd
import json
data = pd.DataFrame(json.loads(open('1token.json','r+').read())).set_index('time')

FileNotFoundError: [Errno 2] No such file or directory: '1token.json'

In [ ]:
from __future__ import division

from datetime import datetime, timedelta
from collections import OrderedDict
from itertools import product
import multiprocessing
import copy

import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 如果安装了seaborn则设置为白色风格
try:
    import seaborn as sns       
    sns.set_style('whitegrid')  
except ImportError:
    pass

from vnpy.trader.vtGlobal import globalSetting
from vnpy.trader.vtObject import VtTickData, VtBarData
from vnpy.trader.vtConstant import *
from vnpy.trader.vtGateway import VtOrderData, VtTradeData


In [ ]:
# encoding: UTF-8

"""
感谢Darwin Quant贡献的策略思路。
知乎专栏原文：https://zhuanlan.zhihu.com/p/24448511

策略逻辑：
1. 布林通道（信号）
2. CCI指标（过滤）
3. ATR指标（止损）

适合品种：螺纹钢
适合周期：15分钟

这里的策略是作者根据原文结合vn.py实现，对策略实现上做了一些修改，仅供参考。

"""

from __future__ import division
from vnpy.trader.vtConstant import *    
from vnpy.trader.app.ctaStrategy.ctaTemplate import (CtaTemplate,
                                                     BarGenerator,
                                                     ArrayManager)


########################################################################
class BitCoinStrategy(CtaTemplate):
    """基于布林通道的交易策略"""
    className = 'BitCoinStrategy'
    author = u'LYF'
    
    TickList=[]
    betaWindow = 120
    betaABSThreshold = 3

    # 策略变量
    beta = 0
    volumeNow = 0
    volume5MinAgo = 0

    intraTradeHigh = 0  # 持仓期内的最高点
    intraTradeLow = 0  # 持仓期内的最低点
    longStop = 0  # 多头止损
    shortStop = 0  # 空头止损

    # 参数列表，保存了参数的名称
    paramList = ['name',
                 'className',
                 'author',
                 'vtSymbol',
                 'betaWindow',
                 'betaThreshold']

    # 变量列表，保存了变量的名称
    varList = ['inited',
               'trading',
               'pos',
               'beta',
               'volumeNow',
               'volume5MinAgo',

               'intraTradeHigh',
               'intraTradeLow',
               'longStop',
               'shortStop']

    # 同步列表，保存了需要保存到数据库的变量名称
    syncList = ['pos',
                'intraTradeHigh',
                'intraTradeLow']

    #----------------------------------------------------------------------
    def __init__(self, ctaEngine, setting):
        """Constructor"""
        super(BitCoinStrategy, self).__init__(ctaEngine, setting)

        self.bg = BarGenerator(self.onTick, 120, self.onTick)  # 创建K线合成器对象
        self.bg120 = BarGenerator(self.onTick, 120, self.on120TickBar())
        self.am = ArrayManager()

    #----------------------------------------------------------------------
    def on120TickBar(self, bar):
        """"""


    #----------------------------------------------------------------------
    def onInit(self):
        """初始化策略（必须由用户继承实现）"""
        self.writeCtaLog(u'%s策略初始化' % self.name)

        # 载入历史数据，并采用回放计算的方式初始化策略数值
        initData = self.loadTick(self.initDays)
        for tick in initData:
            self.onTick(tick)

        self.putEvent()

    #----------------------------------------------------------------------
    def onStart(self):
        """启动策略（必须由用户继承实现）"""
        self.writeCtaLog(u'%s策略启动' % self.name)
        self.putEvent()

    #----------------------------------------------------------------------
    def onStop(self):
        """停止策略（必须由用户继承实现）"""
        self.writeCtaLog(u'%s策略停止' % self.name)
        self.putEvent()

    #----------------------------------------------------------------------
    def onTick(self, tick):
        """收到行情TICK推送（必须由用户继承实现）"""
        #self.bg.updateTick(tick)
        TickList.append(tick)
        
        if len(self.TickList) <= 2:
            return
        else:
            self.TickList.pop(0)
        lastTick = self.TickList[-2]
        
        # 新的一天
        if lastTick.datetime.date() != bar.datetime.date():
            # 如果已经初始化
            if self.dayHigh:
                self.range = self.dayHigh - self.dayLow
                self.longEntry = bar.open + self.k1 * self.range
                self.shortEntry = bar.open - self.k2 * self.range           
                
            self.dayOpen = bar.open
            self.dayHigh = bar.high
            self.dayLow = bar.low

            self.longEntered = False
            self.shortEntered = False
        else:
            self.dayHigh = max(self.dayHigh, bar.high)
            self.dayLow = min(self.dayLow, bar.low)

        # 尚未到收盘
        if not self.range:
            return

        if bar.datetime.time() < self.exitTime:
            if self.pos == 0:
                if bar.close > self.dayOpen:
                    if not self.longEntered:
                        self.buy(self.longEntry, self.fixedSize, stop=True)
                else:
                    if not self.shortEntered:
                        self.short(self.shortEntry, self.fixedSize, stop=True)
    
            # 持有多头仓位
            elif self.pos > 0:
                self.longEntered = True

                # 多头止损单
                self.sell(self.shortEntry, self.fixedSize, stop=True)
                
                # 空头开仓单
                if not self.shortEntered:
                    self.short(self.shortEntry, self.fixedSize, stop=True)
                
            # 持有空头仓位
            elif self.pos < 0:
                self.shortEntered = True

                # 空头止损单
                self.cover(self.longEntry, self.fixedSize, stop=True)
                
                # 多头开仓单
                if not self.longEntered:
                    self.buy(self.longEntry, self.fixedSize, stop=True)
            
        # 收盘平仓
        else:
            if self.pos > 0:
                self.sell(bar.close * 0.99, abs(self.pos))
            elif self.pos < 0:
                self.cover(bar.close * 1.01, abs(self.pos))
 
        # 发出状态更新事件
        self.putEvent()

    #----------------------------------------------------------------------
    def onBar(self, bar):
        """收到Bar推送（必须由用户继承实现）"""
        self.bg.updateBar(bar)

    #----------------------------------------------------------------------
    def onXminBar(self, bar):
        """收到X分钟K线"""
        # 全撤之前发出的委托
        self.cancelAll()

        # 保存K线数据
        am = self.am

        am.updateBar(bar)

        if not am.inited:
            return

        # 计算指标数值
        # self.bollUp, self.bollDown = am.boll(self.bollWindow, self.bollDev)
        # self.cciValue = am.cci(self.cciWindow)
        # self.atrValue = am.atr(self.atrWindow)
        self.beta = 
        self.volumeNow
        self.volume5MinAgo

    # 判断是否要进行交易

    # 当前无仓位，发送开仓委托
    if self.pos == 0:
        self.intraTradeHigh = bar.high
        self.intraTradeLow = bar.low
        
        #做出决策
        if self.cciValue > 0:
            self.buy(self.bollUp, self.fixedSize, True)

        elif self.cciValue < 0:
            self.short(self.bollDown, self.fixedSize, True)

    # 持有多头仓位
    elif self.pos > 0:
        self.intraTradeHigh = max(self.intraTradeHigh, bar.high)
        self.intraTradeLow = bar.low
        self.longStop = self.intraTradeHigh - self.atrValue * self.slMultiplier

        self.sell(self.longStop, abs(self.pos), True)

    # 持有空头仓位
    elif self.pos < 0:
        self.intraTradeHigh = bar.high
        self.intraTradeLow = min(self.intraTradeLow, bar.low)
        self.shortStop = self.intraTradeLow + self.atrValue * self.slMultiplier

        self.cover(self.shortStop, abs(self.pos), True)

    # 同步数据到数据库
    self.saveSyncData()

    # 发出状态更新事件
    self.putEvent()

    #----------------------------------------------------------------------

def onOrder(self, order):
    """收到委托变化推送（必须由用户继承实现）"""
    pass

#----------------------------------------------------------------------
def onTrade(self, trade):
    # 发出状态更新事件
    self.putEvent()

#----------------------------------------------------------------------
def onStopOrder(self, so):
    """停止单推送"""
    pass
